In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost
!pip install optuna

# 0. Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time
import lightgbm as lgb
import catboost as cb
import joblib
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import optuna
from optuna.samplers import TPESampler
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState

# Suprimir os alertas do LightGBM
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

# 1. Leitura dos datasets

In [ ]:
# Note que neste momento importamos as bases contendo todas as variáveis criadas no dataprep

train = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/train_002.csv')
val = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/val_002.csv')

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,1291950,2014-12-29,1,AUTOMOTIVE,1.0,0,53.46,Quito,Pichincha,D,...,3.066667,2.811111,2014,12,29,0,False,53.46,0.018706,44.0
1,1293732,2014-12-30,1,AUTOMOTIVE,4.0,0,54.14,Quito,Pichincha,D,...,3.083333,2.822222,2014,12,30,1,False,216.56,0.073883,40.0
2,1295514,2014-12-31,1,AUTOMOTIVE,0.0,0,53.45,Quito,Pichincha,D,...,3.061111,2.822222,2014,12,31,2,False,0.00,0.000000,43.0
3,1299078,2015-01-02,1,AUTOMOTIVE,2.0,0,52.72,Quito,Pichincha,D,...,3.022222,2.808333,2015,1,2,4,False,105.44,0.037936,44.0
4,1300860,2015-01-03,1,AUTOMOTIVE,6.0,0,50.05,Quito,Pichincha,D,...,3.050000,2.819444,2015,1,3,5,True,250.25,0.099900,43.0


In [ ]:
val.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,2756754,2017-04-01,1,AUTOMOTIVE,9.0,0,50.25,Quito,Pichincha,D,...,3.783333,4.069444,2017,4,1,5,True,552.75,0.218905,79.0
1,2758536,2017-04-02,1,AUTOMOTIVE,4.0,0,50.25,Quito,Pichincha,D,...,3.777778,4.061111,2017,4,2,6,True,552.75,0.218905,76.0
2,2760318,2017-04-03,1,AUTOMOTIVE,11.0,0,50.25,Quito,Pichincha,D,...,3.811111,4.077778,2017,4,3,0,False,552.75,0.218905,66.0
3,2762100,2017-04-04,1,AUTOMOTIVE,3.0,0,50.99,Quito,Pichincha,D,...,3.777778,4.061111,2017,4,4,1,False,152.97,0.058835,66.0
4,2763882,2017-04-05,1,AUTOMOTIVE,5.0,0,51.14,Quito,Pichincha,D,...,3.750000,4.063889,2017,4,5,2,False,255.70,0.097771,65.0


In [ ]:
# Metadados referente ao conjunto de dados
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

generate_metadata(train)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,id,int64,0,0.0,134706
1,sales_rolling_mean_60,float64,0,0.0,61798
2,sales_lag_150,float64,0,0.0,34507
3,sales_lag_180,float64,0,0.0,34486
4,sales_lag_360,float64,0,0.0,33919
5,sales_rolling_mean_3,float64,0,0.0,41647
6,sales_rolling_mean_5,float64,0,0.0,44202
7,sales_rolling_mean_7,float64,0,0.0,46179
8,sales_rolling_mean_14,float64,0,0.0,50659
9,sales_rolling_mean_21,float64,0,0.0,53547


# 2. Avaliar modelo vigente

In [ ]:
# Carregar o modelo salvo
pipeline_vigente = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_model.pkl')

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
train.loc[:,categorical_features] = train[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

# Previsões no treino
train_predictions = pipeline_vigente.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
train_mae = mean_absolute_error(y_train, train_predictions)

# Previsões no teste
test_predictions = pipeline_vigente.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_mae = mean_absolute_error(y_test, test_predictions)

results = []
model_name = 'Modelo Vigente'

results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))

# Mostrar os resultados
results_df1_vigente = pd.DataFrame(results, columns=['Model', 'Train RMSE', 'Test RMSE', 'Train MAE', 'Test MAE'])

results_df1_vigente

,Model,Train RMSE,Test RMSE,Train MAE,Test MAE
0,Modelo Vigente,10948.499005,10910.384667,3395.166383,3287.100141


# 3. Treinar modelos segmentados por Loja

## 3.1 Loja 1

### 3.1.1 Desenvolvimento do modelo segmentado

In [ ]:
# Definir a loja específica
store_nbr = 1

# Filtrar os dados de treino e teste para a loja 1
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# Converter as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# Preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # Armazenar predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# Avaliar os modelos para a loja 1
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# Calcular métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

# Salvar predições em arquivos CSV (opcional)
train_predictions.to_csv('train_predictions_store_1.csv', index=False)
test_predictions.to_csv('test_predictions_store_1.csv', index=False)

Treinando o algoritmo: LinearRegression para a loja 1
Treino finalizado. Tempo gasto no treinamento: 0.64 segundos
RMSE no treino: 1201.4177629885799 
RMSE no teste: 711.9574003760232 
MAE no treino: 387.19457794985516 
MAE no teste: 317.3449131069922 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 1
Treino finalizado. Tempo gasto no treinamento: 10.30 segundos
RMSE no treino: 574.9911555593854 
RMSE no teste: 809.5506315448654 
MAE no treino: 239.89417422851275 
MAE no teste: 348.6044966310326 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 1
Treino finalizado. Tempo gasto no treinamento: 18.21 segundos
RMSE no treino: 967.8352395399995 
RMSE no teste: 529.6064647749638 
MAE no treino: 359.21197782480215 
MAE no teste: 282.4164006177962 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para

### 3.1.2 Comparação com o modelo único (sem segmentação)

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
preds_catboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 1
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 1].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 756.9495448162156
mae = 381.63057569067263


Podemos notar que o melhor modelo segmentado (CatBoost) dentre os avaliados desempenhou muito melhor que o modelo único neste público.  

### 3.1.3 Tuning de hiperparâmetros para o modelo da Loja 1

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros da primeira iteração
initial_params = {
    'iterations': 400,
    'depth': 5,
    'learning_rate': 0.01,
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        iterations = initial_params['iterations']
        depth = initial_params['depth']
        learning_rate = initial_params['learning_rate']

        # Imprimir os valores iniciais
        print(f"Trial 0: iterations={iterations}, depth={depth}, learning_rate={learning_rate}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        iterations = trial.suggest_int('iterations', 200, 2000)
        depth = trial.suggest_int('depth', 3, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = cb.CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=0
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 50 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=50)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=300, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2024-08-25 03:05:50,244] A new study created in memory with name: no-name-34f24ecb-5c4b-49b5-b2aa-8c8397e24d92


Trial 0: iterations=400, depth=5, learning_rate=0.01


[I 2024-08-25 03:06:03,248] Trial 0 finished with value: 529.6064647749638 and parameters: {}. Best is trial 0 with value: 529.6064647749638.
[I 2024-08-25 03:06:31,065] Trial 1 finished with value: 624.1333431565671 and parameters: {'iterations': 435, 'depth': 8, 'learning_rate': 0.010554789041288683}. Best is trial 0 with value: 529.6064647749638.
[I 2024-08-25 03:07:25,983] Trial 3 finished with value: 5422.977405694636 and parameters: {'iterations': 324, 'depth': 9, 'learning_rate': 0.001729574474204545}. Best is trial 0 with value: 529.6064647749638.
[I 2024-08-25 03:07:57,800] Trial 2 finished with value: 790.1290227866684 and parameters: {'iterations': 1284, 'depth': 8, 'learning_rate': 0.03849342037505397}. Best is trial 0 with value: 529.6064647749638.
[I 2024-08-25 03:08:27,009] Trial 4 finished with value: 542.4930939220661 and parameters: {'iterations': 1536, 'depth': 6, 'learning_rate': 0.002554423963679081}. Best is trial 0 with value: 529.6064647749638.
[I 2024-08-25 03:

Melhores hiperparâmetros: {'iterations': 729, 'depth': 7, 'learning_rate': 0.004851134216788116}
Melhor RMSE: 520.7492


### 3.1.4 Treinando o algoritmo com a melhor configuração de hiperparâmetros

In [ ]:
best_params = study.best_params
# best_params = {'iterations': 729, 'depth': 7, 'learning_rate': 0.004851134216788116}

model = cb.CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=0,
    random_state=0
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja1 = pipeline.predict(X_test_store)

# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja1.pkl')

['/content/drive/MyDrive/Previsao_Demandas_PoD_Academy/pipeline_modelo_segmentado_loja1.pkl']

## 3.2 Loja 2

### 3.2.1 Desenvolvimento do modelo segmentado

In [ ]:
# Definir a loja específica
store_nbr = 2

# Filtrar os dados de treino e teste para a loja 2
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# Converter as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# Preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # Armazenar predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# Avaliar os modelos para a loja 2
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# Calcular métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

# Salvar predições em arquivos CSV (opcional)
train_predictions.to_csv('train_predictions_store_2.csv', index=False)
test_predictions.to_csv('test_predictions_store_2.csv', index=False)

Treinando o algoritmo: LinearRegression para a loja 2
Treino finalizado. Tempo gasto no treinamento: 0.26 segundos
RMSE no treino: 4309.902860139099 
RMSE no teste: 1811.4925527945984 
MAE no treino: 861.7047807747417 
MAE no teste: 510.9831774209381 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 2
Treino finalizado. Tempo gasto no treinamento: 3.61 segundos
RMSE no treino: 1752.9254874398048 
RMSE no teste: 5861.086151944521 
MAE no treino: 421.6874406856601 
MAE no teste: 1158.6674070739573 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 2
Treino finalizado. Tempo gasto no treinamento: 5.25 segundos
RMSE no treino: 2155.355780989744 
RMSE no teste: 5608.628089742516 
MAE no treino: 617.9793918578202 
MAE no teste: 892.7441157791701 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a 

### 3.2.2 Comparação com o modelo único (sem segmentação)

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 2
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})
df1 = df[df.store_nbr == 2].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_catboost_loja_2 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1050.5734054269876
mae = 423.77034046931243


Podemos notar que o melhor modelo segmentado (CatBoost) dentre os avaliados não desempenhou melhor que o modelo único neste público.

### 3.2.3 Tuning de hiperparâmetros para o modelo da Loja 2

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler

# Pré-processamento: Escalonamento para variáveis numéricas e One-Hot Encoding para variáveis categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros iniciais
initial_params = {
    'fit_intercept': True,
    'normalize': False  # Parâmetro depreciado, preferimos o StandardScaler
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        fit_intercept = initial_params['fit_intercept']
        normalize = initial_params['normalize']

        # Imprimir os valores iniciais
        print(f"Trial 0: fit_intercept={fit_intercept}, normalize={normalize}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])

    # Definir o modelo com os hiperparâmetros sugeridos
    model = LinearRegression(
        fit_intercept=fit_intercept
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

# Callback para parar o estudo após 50 iterações sem melhorias
class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Criar o estudo Optuna com o sampler TPE
early_stopping = EarlyStoppingCallback(patience=50)
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=300, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2024-08-27 02:19:17,379] A new study created in memory with name: no-name-3cd483f6-9650-4bf0-aa2a-76ed76660522


Trial 0: fit_intercept=True, normalize=False


[I 2024-08-27 02:19:18,099] Trial 1 finished with value: 1811.3855136687266 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:18,145] Trial 0 finished with value: 1811.4925527945984 and parameters: {}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:18,744] Trial 3 finished with value: 1811.4925527945984 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:18,751] Trial 2 finished with value: 1811.4925527945984 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:19,312] Trial 5 finished with value: 1811.3855136687266 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:19,338] Trial 4 finished with value: 1811.4925527945984 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 1811.3855136687266.
[I 2024-08-27 02:19:19,970] T

Melhores hiperparâmetros: {'fit_intercept': False}
Melhor RMSE: 1811.3855


### 3.2.4 Treinando o algoritmo com a melhor configuração de hiperparâmetros

In [ ]:
# Melhores hiperparâmetros encontrados pelo Optuna
best_params = study.best_params

# Definir o modelo LinearRegression com os melhores hiperparâmetros
model = LinearRegression(
    fit_intercept=best_params['fit_intercept']
)

# Criar o pipeline novamente com o preprocessador e o modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Treinar o pipeline com os dados de treino completos
pipeline.fit(X_train_store, y_train_store)

# Fazer previsões nos dados de teste
y_pred_loja2 = pipeline.predict(X_test_store)

# Avaliar o modelo final
final_rmse = mean_squared_error(y_test_store, y_pred_loja2)**0.5
print(f"RMSE final no conjunto de teste: {final_rmse:.4f}")

RMSE final no conjunto de teste: 1811.3855


## 3.1 Loja 3

### 3.3.1 Desenvolvimento do modelo segmentado

In [ ]:
# Definir a loja específica
store_nbr = 3

# Filtrar os dados de treino e teste para a loja 3
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# Converter as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# Preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # Armazenar predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# Avaliar os modelos para a loja 3
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# Calcular métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

# Salvar predições em arquivos CSV (opcional)
train_predictions.to_csv('train_predictions_store_3.csv', index=False)
test_predictions.to_csv('test_predictions_store_3.csv', index=False)

Treinando o algoritmo: LinearRegression para a loja 3
Treino finalizado. Tempo gasto no treinamento: 1.16 segundos
RMSE no treino: 3708.145384198154 
RMSE no teste: 2216.065431634825 
MAE no treino: 1265.3419114327464 
MAE no teste: 984.7348420235331 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 3
Treino finalizado. Tempo gasto no treinamento: 4.49 segundos
RMSE no treino: 1697.3936676906167 
RMSE no teste: 1776.8050370912838 
MAE no treino: 721.2786904789174 
MAE no teste: 878.6855394414252 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 3
Treino finalizado. Tempo gasto no treinamento: 5.26 segundos
RMSE no treino: 3003.511840916979 
RMSE no teste: 1610.2282211101676 
MAE no treino: 1135.928150012923 
MAE no teste: 849.0194848457363 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a

### 3.3.2 Comparação com o modelo único (sem segmentação)

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 3
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})
df1 = df[df.store_nbr == 3].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_catboost_loja_3 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1336.421097365785
mae = 624.2347500299383


### 3.3.3 Tuning de hiperparâmetros para o modelo da Loja 3

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros da primeira iteração
initial_params = {
    'iterations': 400,
    'depth': 5,
    'learning_rate': 0.01,
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        iterations = initial_params['iterations']
        depth = initial_params['depth']
        learning_rate = initial_params['learning_rate']

        # Imprimir os valores iniciais
        print(f"Trial 0: iterations={iterations}, depth={depth}, learning_rate={learning_rate}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        iterations = trial.suggest_int('iterations', 200, 2000)
        depth = trial.suggest_int('depth', 3, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = cb.CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=0
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 50 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=50)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=300, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2024-08-25 16:06:15,277] A new study created in memory with name: no-name-aca1b6dc-ec23-4e44-bd79-37992efb489e


Trial 0: iterations=400, depth=5, learning_rate=0.01


[I 2024-08-25 16:06:29,831] Trial 0 finished with value: 1610.2282211101676 and parameters: {}. Best is trial 0 with value: 1610.2282211101676.
[I 2024-08-25 16:06:53,647] Trial 2 finished with value: 2150.3088621751103 and parameters: {'iterations': 680, 'depth': 5, 'learning_rate': 0.03525433591408741}. Best is trial 0 with value: 1610.2282211101676.
[I 2024-08-25 16:07:33,609] Trial 3 finished with value: 2129.985523855695 and parameters: {'iterations': 1799, 'depth': 4, 'learning_rate': 0.013385872523669571}. Best is trial 0 with value: 1610.2282211101676.
[I 2024-08-25 16:08:33,870] Trial 4 finished with value: 2045.1314169946459 and parameters: {'iterations': 1407, 'depth': 6, 'learning_rate': 0.016525300101372077}. Best is trial 0 with value: 1610.2282211101676.
[I 2024-08-25 16:08:49,503] Trial 5 finished with value: 2163.981038847295 and parameters: {'iterations': 583, 'depth': 4, 'learning_rate': 0.01759451263782257}. Best is trial 0 with value: 1610.2282211101676.
[I 2024-08

### 3.3.4 Treinando o algoritmo com a melhor configuração de hiperparâmetros

In [ ]:
best_params = study.best_params
# best_params = {'iterations': 1073, 'depth': 8, 'learning_rate': 0.0034779983096194914}

model = cb.CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=0,
    random_state=0
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja3 = pipeline.predict(X_test_store)

## 4.1 Loja 4

### 3.4.1 Desenvolvimento do modelo segmentado

In [ ]:
# Definir a loja específica
store_nbr = 4

# Filtrar os dados de treino e teste para a loja 1
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# Converter as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# Preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # Armazenar predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# Avaliar os modelos para a loja 4
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# Calcular métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

# Salvar predições em arquivos CSV (opcional)
train_predictions.to_csv('train_predictions_store_4.csv', index=False)
test_predictions.to_csv('test_predictions_store_4.csv', index=False)

Treinando o algoritmo: LinearRegression para a loja 4
Treino finalizado. Tempo gasto no treinamento: 1.81 segundos
RMSE no treino: 1327.8817713864073 
RMSE no teste: 1022.7538880537892 
MAE no treino: 467.3064904903094 
MAE no teste: 437.93587095181965 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 4
Treino finalizado. Tempo gasto no treinamento: 3.94 segundos
RMSE no treino: 644.2373538112022 
RMSE no teste: 1576.820961924147 
MAE no treino: 284.31308093843825 
MAE no teste: 598.969461478264 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 4
Treino finalizado. Tempo gasto no treinamento: 5.62 segundos
RMSE no treino: 1112.487654843192 
RMSE no teste: 983.4678478221231 
MAE no treino: 436.1676461144851 
MAE no teste: 436.0830642349952 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a 

### 3.4.2 Comparação com o modelo único (sem segmentação)

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 4
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})
df1 = df[df.store_nbr == 4].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1077.4392373912142
mae = 452.7573858835969


### 3.4.3 Tuning de hiperparâmetros para o modelo da Loja 4

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros da primeira iteração
initial_params = {
    'iterations': 400,
    'depth': 5,
    'learning_rate': 0.01,
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        iterations = initial_params['iterations']
        depth = initial_params['depth']
        learning_rate = initial_params['learning_rate']

        # Imprimir os valores iniciais
        print(f"Trial 0: iterations={iterations}, depth={depth}, learning_rate={learning_rate}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        iterations = trial.suggest_int('iterations', 200, 2000)
        depth = trial.suggest_int('depth', 3, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = cb.CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=0
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 30 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=30)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=300, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2024-08-26 03:36:03,641] A new study created in memory with name: no-name-dfadbb8f-7012-455d-922c-7964e075c9ae


Trial 0: iterations=400, depth=5, learning_rate=0.01


[I 2024-08-26 03:36:16,088] Trial 0 finished with value: 983.4678478221231 and parameters: {}. Best is trial 0 with value: 983.4678478221231.
[I 2024-08-26 03:36:41,359] Trial 1 finished with value: 1451.288739319472 and parameters: {'iterations': 1465, 'depth': 5, 'learning_rate': 0.01428437240443262}. Best is trial 0 with value: 983.4678478221231.
[I 2024-08-26 03:36:52,929] Trial 2 finished with value: 825.8794925472876 and parameters: {'iterations': 1635, 'depth': 4, 'learning_rate': 0.0015595787599090919}. Best is trial 2 with value: 825.8794925472876.
[I 2024-08-26 03:36:56,022] Trial 3 finished with value: 764.1225055995284 and parameters: {'iterations': 320, 'depth': 6, 'learning_rate': 0.008910764170025726}. Best is trial 3 with value: 764.1225055995284.
[I 2024-08-26 03:37:45,992] Trial 4 finished with value: 1180.0646517584933 and parameters: {'iterations': 955, 'depth': 7, 'learning_rate': 0.005524236816059419}. Best is trial 3 with value: 764.1225055995284.
[I 2024-08-26 0

### 3.4.4 Treinando o algoritmo com a melhor configuração de hiperparâmetros

In [ ]:
best_params = study.best_params
# best_params = {'iterations': 320, 'depth': 6, 'learning_rate': 0.008910764170025726}

model = cb.CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=0,
    random_state=0
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja4 = pipeline.predict(X_test_store)

# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja4.pkl')

['/content/drive/MyDrive/Previsao_Demandas_PoD_Academy/pipeline_modelo_segmentado_loja4.pkl']

## 5.1 Loja 5

### 3.5.1 Desenvolvimento do modelo segmentado

In [ ]:
# Definir a loja específica
store_nbr = 5

# Filtrar os dados de treino e teste para a loja 5
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# Converter as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# Preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, depth=5, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # Armazenar predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# Avaliar os modelos para a loja 5
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# Calcular métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

# Salvar predições em arquivos CSV (opcional)
train_predictions.to_csv('train_predictions_store_5.csv', index=False)
test_predictions.to_csv('test_predictions_store_5.csv', index=False)

Treinando o algoritmo: LinearRegression para a loja 5
Treino finalizado. Tempo gasto no treinamento: 0.56 segundos
RMSE no treino: 1234.8160080399762 
RMSE no teste: 707.0838822666592 
MAE no treino: 463.397813238673 
MAE no teste: 317.19289405216193 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 5
Treino finalizado. Tempo gasto no treinamento: 5.05 segundos
RMSE no treino: 519.0647337595112 
RMSE no teste: 563.5111736600014 
MAE no treino: 231.57621916365167 
MAE no teste: 263.18433628659284 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 5
Treino finalizado. Tempo gasto no treinamento: 5.21 segundos
RMSE no treino: 939.2965241953054 
RMSE no teste: 623.9853307976273 
MAE no treino: 361.738820866915 
MAE no teste: 286.6869791103264 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a l

### 3.5.2 Comparação com o modelo único (sem segmentação)

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')
preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 4
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})
df1 = df[df.store_nbr == 5].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 647.0688722758842
mae = 328.67539042839866


### 3.5.3 Tuning de hiperparâmetros para o modelo da Loja 5

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Sugerir valores para os hiperparâmetros a partir da segunda iteração
    n_estimators = trial.suggest_int('n_estimators', 200, 2000)
    depth = trial.suggest_int('depth', 3, 9)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        max_depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=42
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 300 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=300)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=300, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2024-08-26 19:01:06,116] A new study created in memory with name: no-name-48dea57c-7da8-4234-a7f8-c31d01bbf00a
[I 2024-08-26 19:01:07,975] Trial 0 finished with value: 7465.64191446895 and parameters: {'n_estimators': 655, 'depth': 7, 'learning_rate': 0.0016874646978118064}. Best is trial 0 with value: 7465.64191446895.
[I 2024-08-26 19:01:07,999] Trial 1 finished with value: 1294.5359460061904 and parameters: {'n_estimators': 1209, 'depth': 7, 'learning_rate': 0.018405149893112427}. Best is trial 1 with value: 1294.5359460061904.
[I 2024-08-26 19:01:09,794] Trial 3 finished with value: 7046.0455861927285 and parameters: {'n_estimators': 1764, 'depth': 7, 'learning_rate': 0.0022612941871551304}. Best is trial 1 with value: 1294.5359460061904.
[I 2024-08-26 19:01:09,817] Trial 2 finished with value: 5546.249777237083 and parameters: {'n_estimators': 1214, 'depth': 3, 'learning_rate': 0.004621982407748883}. Best is trial 1 with value: 1294.5359460061904.
[I 2024-08-26 19:01:11,720] Tr

Melhores hiperparâmetros: {'n_estimators': 1100, 'depth': 5, 'learning_rate': 0.034769109023931925}
Melhor RMSE: 514.4250


### 3.5.4 Treinando o algoritmo com a melhor configuração de hiperparâmetros

In [ ]:
best_params = study.best_params
# best_params = {'n_estimators': 1100, 'depth': 5, 'learning_rate': 0.034769109023931925}

model = lgb.LGBMRegressor(
    n_estimators=best_params['n_estimators'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=-1,
    random_state=42
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja5 = pipeline.predict(X_test_store)

# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_modelo_segmentado_loja5.pkl')

['/content/drive/MyDrive/Previsao_Demandas_PoD_Academy/pipeline_modelo_segmentado_loja5.pkl']

## 4. Avaliação dos resultados no público completo

### 4.1. Avaliação técnica do modelo único

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/pipeline_best_model_all_features_catboost.pkl')

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
train.loc[:,categorical_features] = train[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})

rmse_modelo_unico = np.sqrt(mean_squared_error(df['target'], df['pred']))
mae_modelo_unico = mean_absolute_error(df['target'], df['pred'])

print(f'RMSE = {rmse_modelo_unico}')
print(f'mae = {mae_modelo_unico}')

RMSE = 1004.2182908920599
mae = 442.2136885003837


### 4.2. Avaliação técnica dos modelos segmentados

In [ ]:
result = np.hstack((y_pred_loja1, preds_catboost_loja_2, preds_catboost_loja_3, y_pred_loja4, y_pred_loja5))

rmse_modelo_segmentado = np.sqrt(mean_squared_error(yreal, result))
mae_modelo_segmentado = mean_absolute_error(yreal, result)

print(f'rmse = {rmse_modelo_segmentado}')
print(f'mae = {mae_modelo_segmentado}')

rmse = 916.8298885321577
mae = 417.7926691929652


In [ ]:
df_metrics = pd.DataFrame({
    'Modelo': ['Único', 'Segmentado'],
    'RMSE': [rmse_modelo_unico, rmse_modelo_segmentado],
    'MAE': [mae_modelo_unico, mae_modelo_segmentado]
})

df_metrics

,Modelo,RMSE,MAE
0,Único,1004.218291,442.213689
1,Segmentado,916.829889,417.792669
